In [ ]:
import tiktoken
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

#####################################
# Chapter 2: 데이터 로딩 및 처리
#####################################

class GPTDatasetV1(Dataset):
    """
    GPT 학습을 위한 데이터셋 클래스입니다.
    텍스트를 입력받아 토큰화하고, 입력(input)과 타겟(target) 쌍을 만듭니다.
    """
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []
                
        # 1. 전체 텍스트를 토큰화합니다.
        # <|endoftext|> 같은 특수 토큰도 허용하여 인코딩합니다.
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        
        # 2. 슬라이딩 윈도우 방식으로 데이터를 조각냅니다.
        # stride만큼 이동하면서 max_length 길이의 덩어리(chunk)를 만듭니다.
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1: i+max_length+1]
            
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
            
            
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self,idx):
        return self.input_ids[idx], self.target_ids[idx]
    
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last = True, num_workers=0):
    """
    텍스트 데이터를 받아 학습에 사용할 DataLoader를 생성하는 함수입니다.
    """
    # 토크나이저 초기화 (GPT-2용 BPE 인코딩 사용)
    tokenizer = tiktoken.get_encoding("gpt2")
    
    # 데이터셋 생성
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    
     # 데이터로더 생성 (배치 단위로 데이터를 묶어주고 셔플링 수행)
    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last = drop_last, num_workers=num_workers)
    
    return dataloader
                         
            
                                     
                                
        
        

In [ ]:
#####################################
# Chapter 3: 어텐션 메커니즘 (모델의 핵심) // Exercise_Attention에 있는 MultiHeadAttention(nn.Module) 과 동일
#####################################
class MultiHeadAttention(nn.Module):
    """
    멀티 헤드 셀프 어텐션 (Multi-Head Self-Attention) 모듈입니다.
    입력 데이터 간의 관계성을 여러 관점(Head)에서 병렬로 학습합니다.
    """
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "출력 차원(d_out)은 헤드 수(num_heads)로 나누어 떨어져야 합니다."

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads  # 각 헤드가 담당할 차원 크기

        # Query, Key, Value를 만들기 위한 선형 투영 레이어들
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        
        # 멀티 헤드 결과를 하나로 합친 후 통과시키는 출력 레이어
        self.out_proj = nn.Linear(d_out, d_out) 
        self.dropout = nn.Dropout(dropout)
        
        # Causal Mask (인과적 마스킹) 생성: 미래의 토큰을 보지 못하게 함
        # 상삼각 행렬(대각선 위쪽)을 1로 채워서 나중에 마스킹에 사용
        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape # b: 배치 크기, num_tokens: 시퀀스 길이

        # 1. Q, K, V 계산
        keys = self.W_key(x)     # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # 2. 헤드 나누기 (Multi-head splitting)
        # 차원을 변형하여 여러 헤드가 병렬로 처리할 수 있게 함
        # (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # 3. 차원 순서 변경 (Transpose)
        # (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        # 이렇게 하면 (num_tokens, head_dim) 행렬이 헤드 개수만큼 독립적으로 존재하게 됨
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # 4. Scaled Dot-Product Attention 계산
        # Query와 Key의 내적 (유사도 계산)
        attn_scores = queries @ keys.transpose(2, 3)  # 결과 Shape: (b, num_heads, num_tokens, num_tokens)

        # 5. 마스킹 (Masking)
        # 현재 시점보다 미래의 토큰 정보를 참조하지 못하게 -inf로 가림
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        # 6. 소프트맥스 및 드롭아웃
        # 점수를 확률로 변환 (합이 1이 되도록)
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # 7. Value와의 가중치 합 (Context Vector 계산)
        # Shape: (b, num_heads, num_tokens, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 

        # 8. 헤드 결합 (Concatenation)
        # 나눠졌던 헤드들을 다시 원래의 d_out 차원으로 합침
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        
        # 9. 최종 선형 투영
        context_vec = self.out_proj(context_vec) 

        return context_vec
